# Quick & Dirty e2e pipeline

## Test NotionDB connection
ref: https://python.langchain.com/docs/integrations/document_loaders/notiondb

In [1]:
!python -V

Python 3.11.7


In [2]:
from langchain_community.document_loaders import NotionDBLoader
import tomllib

In [3]:
with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

with open('../.notion_databases.toml', 'rb') as f:
    _DATABASES_NOTION = tomllib.load(f)

In [4]:
_DATABASES_NOTION

{'笔记（非文学）': '7443174d151342458fb7e47acdbb0c64',
 '读书笔记（文学）': '808e50735cce4c66a151f3e8b79c8d9d',
 '写作': '8b81c07089344d8d95ddba22bafb8c12',
 '格言小集': '04d75f24b9544f3f8b220d5e7d87b7c3'}

In [5]:
loader = NotionDBLoader(
    integration_token=_TOKENS['notion'],
    database_id=_DATABASES_NOTION['格言小集'],
    request_timeout_sec=30,  # optional, defaults to 10
)

In [6]:
%%time
docs = loader.load()

CPU times: user 180 ms, sys: 20.9 ms, total: 201 ms
Wall time: 5.71 s


In [7]:
print(docs[:2])

[Document(page_content='人生四不捡：塔吊下边冰红茶，铁轨上边牛肉干，过山车下八宝粥，课桌后边葡萄干', metadata={'name': '笑死', 'id': '2303056b-72f1-4f9a-b377-cf6b12a0d424'}), Document(page_content='\n名为太阳的溶剂\n\n楼群之中，乌鸦堕落为一种失去预言性的可厌生物\n\n对大多数人来说，他们所抱定的观念只是为他们生存方式做辩护的使用说明书而已。\n\n颤抖的彩虹其实是光的边角余料。\n\n以前的建筑是不透明的甲壳类，现在的建筑是透光的腔肠类。\n\n射电望远镜就是宇宙射线的向日葵。\n\n电线像静脉一样生长着。\n\n绝望的尽头并非对绝望感到绝望，而是对绝望感到厌烦。相对地，希望的尽头也只是对希望的疲倦而已。与一般的见解不同，绝望和希望是生活的激发态。\n', metadata={'name': '沃滋集朔德', 'id': '340f4781-1042-4534-b286-b758b0ca09a4'})]


## Conversational RAG with memory
https://python.langchain.com/docs/expression_language/cookbook/retrieval

- llm-aided retrieval
- ReAct framework

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [9]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=_TOKENS['huggingface'], 
    # https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2#sentence-transformersparaphrase-multilingual-minilm-l12-v2
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

In [10]:
%%time
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings, 
    persist_directory='../database/playground_test'
)

# TODO: use SelfQueryRetriever to allow in metadata context
# https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/
retriever = vectorstore.as_retriever()

CPU times: user 313 ms, sys: 46.2 ms, total: 359 ms
Wall time: 551 ms


In [26]:
# # this uses huggingfacehub to compute
# # free but can be very slow
# from langchain import HuggingFaceHub

# llm = HuggingFaceHub(
#     huggingfacehub_api_token=_TOKENS['huggingface'],
#     repo_id='01-ai/Yi-6B-Chat', 
#     # https://huggingface.co/transformers/v4.9.2/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
#     model_kwargs={"temperature": 0.1},
# )

In [30]:
# use local model
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

In [ ]:
!ollama serve

In [32]:
import langchain
langchain.debug = True

In [36]:
langchain.debug = False

In [38]:
%%time 
from langchain.prompts import PromptTemplate, ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])
chain = prompt | llm 

chain.invoke({"input": "who are you?"})

CPU times: user 19.3 ms, sys: 6.06 ms, total: 25.4 ms
Wall time: 9.57 s


"\nAh, a curious human! *adjusts glasses* I am a world-class technical documentation writer, here to enlighten you on the intricacies of documenting complex systems in a clear and concise manner. My expertise spans a wide range of industries, from software development to medical devices, and everything in between.\n\nAs a technical writer, my job is to create user manuals, guides, and other documentation that help users understand how to use a product or system. I work closely with engineers, designers, and other stakeholders to ensure that the documentation is accurate, up-to-date, and meets the needs of the target audience.\n\nMy toolkit includes a variety of technical writing tools, such as MadCap Flare, Adobe FrameMaker, and Confluence. I am well-versed in creating modular, reusable content that can be easily updated and maintained over time. My documentation styles are clean, consistent, and easy to follow, with clear headings, concise language, and intuitive navigation.\n\nIn add

In [39]:
from operator import itemgetter

from langchain.schema import format_document
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

from langchain.memory import ConversationBufferMemory

from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.output_parsers import StrOutputParser

# prompt template
# https://github.com/langchain-ai/langchain/tree/master/templates

In [44]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [49]:
_template = """请将以下对话翻译成中文，并将后续问题以原语言改写为一个新的独立的问题。


对话记录：
{chat_history}
后续问题：{question}
新的问题："""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)


DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


template = """请仅仅根据以下资料回答问题。请用中文回答：
{context}

问：{question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [50]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}

# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [52]:
%%time

inputs = {"question": '"勤钳工，懒车工"后面是什么？'}
result = final_chain.invoke(inputs)
result

CPU times: user 54.9 ms, sys: 12.2 ms, total: 67.1 ms
Wall time: 23.2 s


{'answer': " Based on the quotes you provided, here are some new questions that can be asked:\n\n1. How do you think people's personalities are shaped by their experiences and environments?\n2. What do you think is the most important quality for a person to have in order to achieve their goals?\n3. In your opinion, what is the key to success in life?\n4. How do you think people can overcome their fears and insecurities to reach their full potential?\n5. What do you think is the biggest challenge that people face in their daily lives, and how can they overcome it?\n6. In your opinion, what is the most difficult thing for a person to change about themselves?\n7. How do you think people can maintain their motivation and drive to pursue their passions and interests over time?\n8. What do you think is the biggest source of happiness and fulfillment in life?\n9. In your opinion, what is the most important thing for a person to learn in order to live a meaningful and fulfilling life?\n10. How

In [ ]:
prompt_template = """
您是一款人工智能语言模型助手。您的任务是根据以下相关资料找到问题的答案。如果你不知道答案，请问答“我不知道。”

资料：{context}

问：{query}
答：
"""